네이버쇼핑 크롤링 코드입니다. 


검색어와 검색하고자 하는 페이지를 입력하면 크롤링합니다.

In [46]:
import re, requests, csv
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os

In [54]:
#크롬드라이버가 있는곳의 경로를 chromedriver까지 써줍니다.
path = os.getcwd().replace('\\','/', 10)
driver_path = path + '/chromedriver_win32/chromedriver'

#### 스크롤 함수 

네이버 쇼핑은 스크롤 하기 전에는 상위 5개 항목만 인식합니다. 

따라서 스크롤 해주는 함수가 필요합니다.

In [30]:
#출처 : https://hello-bryan.tistory.com/194
def scroll_page(driver, scroll_pause_sec):
    """driver, 스크롤 후 정지 초를 넘겨주면 맨 아래까지 스크롤해주는 함수"""
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(scroll_pause_sec)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

#### 크롤러
드라이버 패스를 넘겨주면 검색어, 페이지를 입력받아 리뷰와 쇼핑몰 판매정보를 크롤링합니다. 
 <br>
 <br>
 _price_info, reviews_ 는 딕셔너리 형태이며
크롤러는 다음 튜플을 반환합니다.
 ```(price_info, reviews)```
 
__price_info__

|gift |	mall | name	| price | shipping | title
---|---|---|---|---|---|
1||||||
2||||||



__reviews__

|content |	date | id	| mall | name | option | title
---|---|---|---|---|---|---|
1|||||||
2|||||||


In [ ]:
def naver_shopping_crawler(driver_path):

    base_url = 'https://shopping.naver.com/'

    print('검색어를 입력해주세요 : ')
    word = input()

    print('로딩중')

    driver = webdriver.Chrome(driver_path)
    driver.get(base_url)
    print('...')
    time.sleep(2)
    
    #검색어 검색하기
    elem = driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/input[1]')
    elem.send_keys(word)
    elem.send_keys(Keys.RETURN)
    print('...')
    time.sleep(2)
    
    #가격비교 클릭
    #상품별로 검색할때 xpath가 다르므로 수정 필요
    compare_price = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/ul/li[2]/a')
    compare_price.click()
    print('...')
    time.sleep(2)

    #리뷰순으로 보기 클릭
    #상품별로 검색할때 xpath가 다르므로 수정 필요
    sorted_by_review_count = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/div/div[1]/a[5]')
    sorted_by_review_count.click()
    print('...')
    time.sleep(2)

    #페이지 맨 아래로 스크롤
    scroll_page(driver, 1)
    
    #페이지 정보 받아와서 상품별 url 리스트로 만듦
    html_list = driver.page_source 
    soup_list = BeautifulSoup(html_list)#, 'html.parser')

    url_list = soup_list.find_all('a', {'class':"basicList_link__1MaTN"})
    print('...')
    time.sleep(3)
    print("1페이지  제품 수 : " +str(len(url_list)))

    #가격비교에 있는 상품 수
    price_compare_count = soup_list.find_all('span', {'class':'subFilter_num__2x0jq'})[1]
    price_compare_count = int(re.sub('<.*?>|,','', str(price_compare_count)))

    if price_compare_count % 40 != 0:
        price_compare_pages = (price_compare_count //40) +1
    else:
        price_compare_pages = (price_compare_count //40)


    print("최대 수집가능 페이지 수 : ", str(price_compare_pages),'\n','검색을 원하는 페이지 수를 입력해주세요')
    n_pages = int(input())
    print('페이지 하나 당 위에서부터 가져올 제품 개수를 입력해주세요')
    n_product = int(input())

    assert n_pages <= price_compare_pages, "페이지 수 초과"

    price_info = []
    ap_0 = price_info.append

    reviews = []
    ap_1 = reviews.append

    num1 = 0

    for page in range(1,1+n_pages):


        num1+=1 #페이지 카운트
        print('--------------------'+str(num1)+'페이지 입니다----------------------\n')
        num2 = 0 #제품 카운트

        for url in tqdm(url_list[:n_product+1]):
            url_i = re.search(r'href=[\'"]?([^\'" >]+)', str(url), re.I|re.S).group(0)[6:]
            driver.get(url_i)
            html_product_i = driver.page_source
            soup_i=BeautifulSoup(html_product_i)

            try:
                review_count = soup_i.find_all('a', {'href':'#section_review'})[0]
                review_count = int(re.sub(r'[^0-9]','', str(review_count)))

            except:
                review_count = 0
            #print(review_count)

            name = soup_i.find('div', {'class':'h_area'})
            name = re.sub('<.+?>','', str(name.find('h2')), re.I|re.S).strip()

            #print(name)

            #제품 카운트
            num2+=1
            print(str(num1)+'페이지 '+str(num2)+'번 제품입니다\n')

            # 판매자 이름
            
            try:
                price_list = soup_i.find_all('table', {'class':'tbl tbl_v'})
            except:
                print("이 상품은 품절입니다.")
                driver.back()
                continue

            for price_i in price_list:

                price_mall = price_i.find('a', {'class':'_priceListMallLogo'})
                price_mall = re.search('data-mall-name=\".*?\"', str(price_mall), re.I|re.S).group(0)[16:-1]

                price_title = price_i.find('a',{'class':'goods_tit'})
                price_title = re.sub("<.*?>", '', str(price_title))
                price_title = re.sub("\n", '', str(price_title)).strip()

                gift = price_i.find('p',{'class':'goods_gift'})
                gift = re.sub("<.*?>", '', str(gift))
                gift = re.sub("\n", '', str(gift)).strip()

                price = price_i.select('td > a > span')[-1]
                price = int(re.sub("<.*?>|,", '', str(price)))

                shipping = price_i.select('td > p')[1]
                shipping = re.sub("<.*?>|,", '', str(shipping))


                price_info_i = {'name':name,
                               'mall':price_mall,
                               'title':price_title,
                               'gift':gift,
                               'price':price,
                               'shipping':shipping}

                ap_0(price_info_i)
            
            #총 리뷰 페이지 수
            #리뷰는 한페이지에 20개까지 표시됩니다.
            if review_count == 0:
                print("이 제품은 리뷰가 없습니다. \n")
                driver.back()
                continue
            if review_count % 20 != 0:
                review_page = (review_count // 20) + 1
            else:
                review_page = (review_count // 20)


            for page in tqdm(range(1, review_page+1)):

                time.sleep(2)

                html_for_rvw = driver.page_source
                soup_for_rvw = BeautifulSoup(html_for_rvw, 'html.parser')
                
                
                #해당 페이지에 현재 보이는 리뷰들의 리스트
                review_list = soup_for_rvw.find('ul', {'id':'_review_list'})
                review_list = review_list.find_all('div', {'class':'atc_area'})

                for review in review_list:

                    rvw_info = review.find_all('span', {'class':'info_cell'})
                    mall = re.sub('<.+?>', '', str(rvw_info[0]), re.I|re.S)
                    id_ =  re.sub('<.+?>', '', str(rvw_info[1]), re.I|re.S)
                    date = re.sub('<.+?>', '', str(rvw_info[2]), re.I|re.S)
                    try: #상품 옵션은 있을 수도 없을 수도 있습니다.
                        option = re.sub('<.+?>', '', str(rvw_info[3]), re.I|re.S)
                    except:
                        option = ''

                    rvw_title = review.find('p', {'class':'subjcet'})
                    rvw_title = re.sub('<.+?>', '', str(rvw_title), re.I|re.S)

                    rvw_content = review.find('div', {'class':'atc'})
                    rvw_content = re.sub('<.+?>', '', str(rvw_content), re.I|re.S)
                    
                    #리뷰정보를 딕셔너리로 저장
                    rvw_dic = {'name':name,
                                'mall' :mall,
                               'id' : id_,
                               'date' : date,
                               'option' :option,
                               'title' :rvw_title,
                               'content' :rvw_content}
                    ap_1(rvw_dic)
                
                #현재리뷰페이지가 마지막이아니라면, 다음리뷰페이지 버튼을 클릭합니다.
                if page < review_page:
                    path = '//a[@onclick="shop.detail.ReviewHandler.page({}, \'_review_paging\'); return false;"]'.format(page+1)
                    button = driver.find_element_by_xpath(path)
                    button.send_keys(Keys.ENTER)

            driver.back()

        #현재 페이지가 지정한 페이지 수보다 적다면, 다음 페이지로 가요
        if page < n_pages:
            next_page_path = '//a[@class="pagination_next__1ITTf"]'
            driver.find_element_by_xpath(next_page_path).click()
            time.sleep(2)
            scroll_page(driver, 1)

    driver.close()
    
    return (price_info,reviews)

In [35]:
#함수 실행
naver_shopping_crawler(driver_path)

검색어를 입력해주세요 : 
쪽쪽이 살균
로딩중
40
최대 수집가능 페이지 수 :  2 
 검색을 원하는 페이지 수를 입력해주세요
1
페이지 하나 당 위에서부터 가져올 제품 개수를 입력해주세요
40
--------------------1페이지 입니다----------------------



1페이지 1번 제품입니다




1페이지 2번 제품입니다




1페이지 3번 제품입니다




1페이지 4번 제품입니다



...


1페이지 6번 제품입니다




1페이지 7번 제품입니다




1페이지 8번 제품입니다




1페이지 9번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 10번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 11번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 12번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 13번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 14번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 15번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 16번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 17번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 18번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 19번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 20번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 21번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 22번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 23번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 24번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 25번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 26번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 27번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 28번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 29번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 30번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 31번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 32번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 33번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 34번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 35번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 36번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 37번 제품입니다

이 제품은 리뷰가 없습니다. 

1페이지 38번 제품입니다

([{'name': '쪼비 쪽쪽이 살균 케이스',
   'mall': 'SSG닷컴',
   'title': '★리뉴얼+뉴컬러 런칭★ 쪼비 -  쪽쪽이 살균 케이스',
   'gift': '네이버 쇼핑 최대 5%  할인쿠폰 [씨티카드7%할인가 : 41,065 원]',
   'price': 44156,
   'shipping': '2500원'},
  {'name': '쪼비 쪽쪽이 살균 케이스',
   'mall': '신세계몰',
   'title': '★리뉴얼+뉴컬러 런칭★ 쪼비 -  쪽쪽이 살균 케이스',
   'gift': '[씨티카드7%할인가 : 43,226 원]',
   'price': 46480,
   'shipping': '2500원'},
  {'name': '쪼비 쪽쪽이 살균 케이스',
   'mall': '쪼비',
   'title': '[쪼비공식몰] 해피추석 쪼비 10% 할인 + 무료배송',
   'gift': '',
   'price': 50400,
   'shipping': '무료배송'},
  {'name': '쪼비 쪽쪽이 살균 케이스',
   'mall': '텐바이텐',
   'title': '쪼비_쪽쪽이 살균 소독 케이스 살균용품',
   'gift': '신규 앱다운시 5천원 할인쿠폰/ 신규회원가입시 5천원,1만원,3만원 할인쿠폰 증정',
   'price': 53200,
   'shipping': '2500원'},
  {'name': '쪼비 쪽쪽이 살균 케이스',
   'mall': 'HAGO',
   'title': 'JJOBI [★김나영PICK]쪽쪽이 살균소독 케이스 쪼비_핑크',
   'gift': '가입 즉시 VIP! 상시 10% 할인 혜택',
   'price': 53200,
   'shipping': '2500원'},
  {'name': '큐티톡 휴대용 쪽쪽이 살균 소독기',
   'mall': '11번가',
   'title': '큐티톡 시그니처 휴대용 쪽쪽이 젖병 살균 소독기',
   'gift': '[삼성카드 1% 청구할인

In [36]:
#실행결과를 data변수에 저장하려면
data = naver_shopping_crawler(driver_path)

검색어를 입력해주세요 : 
쪽쪽이 살균
로딩중
40
최대 수집가능 페이지 수 :  2 
 검색을 원하는 페이지 수를 입력해주세요
1
페이지 하나 당 위에서부터 가져올 제품 개수를 입력해주세요
8
--------------------1페이지 입니다----------------------



1페이지 1번 제품입니다




1페이지 2번 제품입니다




1페이지 3번 제품입니다




1페이지 4번 제품입니다




1페이지 5번 제품입니다




1페이지 6번 제품입니다




1페이지 7번 제품입니다




1페이지 8번 제품입니다




1페이지 9번 제품입니다

이 제품은 리뷰가 없습니다. 




In [56]:
naver_reviews = pd.DataFrame(data[1])
naver_reviews.to_csv("naver_reiviews")
naver_reviews.tail()

,content,date,id,mall,name,option,title
2112,휴대하기 편리하며 다양한 소독용도 활용 및 여행 시 외출 시 젖병.아이 소형 장난감...,20.05.21.,elle****,핸디트립,휴대용 젖병 쪽쪽이 소독기 미니 공갈꼭지 살균기,,휴대하기 편리하며 다양한 소독용도 활용 및 여행 시 외출 시 젖병.아이 소형 장난감...
2113,친구랑 만날일이있어 잠시 만났는데 친구 애기가 마스크 밑에 쪽쪽이를 하고있더라구요 ...,20.08.24.,asr*****,위메프,미니짱 쪽쪽이 공갈노리개 휴대용 자외선살균기 ns-30,,친구랑 만날일이있어 잠시 만났는데 친구 애기가 마스크 밑에 쪽쪽이를 하고있더라구요 ...
2114,아무거나 잘들어가서 좋아요살균이 되는건가 싶은데 뚜껑열면 멈추니 확인할방법이 없잖아...,20.09.09.,zlzl****,미니짱,미니짱 쪽쪽이 공갈노리개 휴대용 자외선살균기 ns-30,,아무거나 잘들어가서 좋아요살균이 되는건가 싶은데 뚜껑열면 멈추니 확인할방법이 없잖아...
2115,쪽쪽이 살균기 용도로 구입했는데 가장 좋은건충전식이라 건전지가 필요없구 사용하기도 ...,20.07.31.,i*g*****,G마켓,미니짱 쪽쪽이 공갈노리개 휴대용 자외선살균기 ns-30,,마음에 쏙~~~드는 소독기 입니다
2116,편리하게 쓰여요 좋습니다,20.05.31.,h*t*****,G마켓,휴대용 젖병소독기 쪽쪽이 치발기 살균 케이스 UV살균기 큐티톡,,미니사이즈


In [55]:
naver_price_info  = pd.DataFrame(data[0])
naver_price_info.to_csv("naver_price_info.csv")
naver_price_info.tail()

,gift,mall,name,price,shipping,title
44,[삼성카드 1% 청구할인] [신한/KB국민/현대 100만원 이상 SK pay 결제 ...,11번가,휴대용 젖병소독기 쪽쪽이 치발기 살균 케이스 UV살균기 큐티톡,30000,무료배송,휴대용 젖병소독기 쪽쪽이 치발기 살균 케이스 UV살균기 큐티톡 시그니처
45,[KB국민/현대/신한 스마일페이 100만원 이상 결제 시 최대 20개월 무이자][삼...,G마켓,휴대용 젖병소독기 쪽쪽이 치발기 살균 케이스 UV살균기 큐티톡,30000,무료배송,휴대용 젖병소독기 쪽쪽이 치발기 UV살균기 큐티톡
46,"신규 앱다운시 5천원 할인쿠폰/ 신규회원가입시 5천원,1만원,3만원 할인쿠폰 증정",텐바이텐,NO.1육아필수템 휘아 츄케어 쪽쪽이살균기,39750,2500원,[NO.1육아필수템] 휘아 츄케어(쪽쪽이살균기) 살균용품
47,첫 구매 시 30일 동안 무제한 무료배송 / 로켓배송은 오늘 주문 내일 도착 / 로...,쿠팡,NO.1육아필수템 휘아 츄케어 쪽쪽이살균기,39750,2500원,[NO.1육아필수템] 휘아 츄케어(쪽쪽이살균기)
48,[삼성카드 1% 청구할인] [신한/KB국민/현대 100만원 이상 SK pay 결제 ...,11번가,NO.1육아필수템 휘아 츄케어 쪽쪽이살균기,39750,2500원,[NO.1육아필수템] 휘아 츄케어(쪽쪽이살균기)


#### 수정 사항
+ 크롤링 원하는 상품 +1이 되는 문제


+ compare_price 부분 수정 

    + //*[@id="__next"]/div/div[2]/div[2]/div[4]/div[1]/div[1]/ul/li[2]/a
    + 검색어마다 구조가 다름
    
    
+ url_list도 저기 걸리는 태그가 검색어마다 다름


+ 스크롤 내리는거 수정(수정완료)


+ 로딩시간 기다리기 표시